# GCC128 - Inteligência Artificial

## Trabalho de implementação do algoritmo KNN com o dataset iris

Dupla:

- Lucas Antonio Lopes Neves (201720357)

- Vinicius Tavares Pimenta (201720354)

### Importação das bibliotecas

In [2]:
import pandas as pd

### Classe que representa o algoritmo

In [6]:
class Algorithm:
    """
    Classe que representa o algoritmo KNN a ser executado sobre a populacao
    de individuos. 
    """
    def __init__(self, k):
        """
        Metodo construtor da classe Algorithm. Executa todos os passos do
        algoritmo.
        
        Passos:
            1) Leitura dos dados.
            2) Calculo das distancias.
            3) Classificacao dos individuos.
            4) Calculo da matriz de confusao.
            5) Calculo da taxa de representacao.
            6) Exibicao dos resultados.
        """
        self.k = k
        
        self.read_data()

        self.calculate_distances()
        
        self.classify_individuals()
        
        self.calculate_confusion_matrix()
        
        self.calculate_accuracy()
        
        self.show_results()

    @classmethod
    def calculate_euclidean_distance(cls, individual_1, individual_2):
        """
        Metodo que calcula a Distancia Euclidiana entre dois individuos.
        
        Args:
            individual_1: Row do dataframe que representa o primeiro individuo.
            individual_2: Row do dataframe que representa o segundo individuo.
            
        Returns:
            float: Distancia Euclidiana entre os individuos.
        """
        pow_sepal_length = pow(individual_1.SepalLengthCm - individual_2.SepalLengthCm, 2)

        pow_sepal_width = pow(individual_1.SepalWidthCm - individual_2.SepalWidthCm, 2)

        pow_petal_length = pow(individual_1.PetalLengthCm - individual_2.PetalLengthCm, 2)

        pow_petal_width = pow(individual_1.PetalWidthCm - individual_2.PetalWidthCm, 2)

        return pow(pow_sepal_length + pow_sepal_width + pow_petal_length + pow_petal_width, 1/2)
    
    def read_data(self):
        """
        Metodo que realiza a leitura dos dados.
        """
        self.population = pd.read_csv('../data/iris.csv')
    
    def calculate_distances(self):
        """
        Metodo que calcula as distancias.
        """
        self.distances = pd.DataFrame(index = self.population.Id)

        for row_1 in self.population.itertuples():
            distances = []

            for row_2 in self.population.itertuples():
                distances.append(Algorithm.calculate_euclidean_distance(row_1, row_2))
            
            self.distances.insert(len(self.distances.columns), row_1.Id, distances)
            
    def get_the_ids_of_k_nearest_neighbors_by_id(self, individual_id):
        """
        Metodo que busca os IDs dos k vizinhos mais proximos pelo ID do individuo.
        
        Args:
            individual_id (int): ID do individuo.
        
        Returns:
            list of int: Lista com os IDs dos k vizinhos mais proximos.
        """
        return self.distances[individual_id].sort_values().index[1:self.k+1]
    
    def get_specie_by_id(self, individual_id):
        """
        Metodo que busca a especie do individuo a partir do seu ID.
        
        Args:
            individual_id (int): ID do individuo.
        
        Returns:
            str: Especie do individuo.
        """
        return self.population.iloc[individual_id-1]['Species']
        
    def classify_individuals(self):
        """
        Metodo para classificar os individuos. A partir do metodo apply do
        dataframe, percorre todos os individuos e classifica cada individuo
        com a moda da especies de seus k vizinhos mais proximos.
        """
        def classify_individual(individual):
            """
            Funcao auxiliar que classifica o individuo.
            
            Args:
                individual: Individuo a ser classificado.
            
            Returns:
                str: Especie classificada para o individuo.
            """
            neighbors = self.get_the_ids_of_k_nearest_neighbors_by_id(individual.Id)
            
            species = [self.get_specie_by_id(neighbor) for neighbor in neighbors]
            
            return pd.Series(species).mode()[0]
        
        self.population['Classification'] = self.population.apply(
            lambda row: classify_individual(row),
            axis = 1
        )
        
    def calculate_confusion_matrix(self):
        """
        Metodo para calcular a matriz de confusao.
        """
        self.confusion_matrix = pd.DataFrame(
            index = self.population.Species.unique()
        )
        
        for specie_real in self.confusion_matrix.index:
            data_filted = self.population.loc[self.population['Species'] == specie_real]
            
            data = []
            
            for specie_classificated in self.confusion_matrix.index:
                if specie_classificated in data_filted['Classification'].value_counts():
                    data.append(data_filted['Classification'].value_counts()[specie_classificated])
                else:
                    data.append(0)
            
            self.confusion_matrix.insert(
                len(self.confusion_matrix.columns),
                specie_real,
                data
            )
            
    def calculate_accuracy(self):
        """
        Metodo para calcular a taxa de reconhecimento.
        """
        classes = self.confusion_matrix.index
        
        data = 0
        
        for classe in classes:
            data += self.confusion_matrix.loc[classe][classe]
            
        self.accurancy = round(data / self.population.shape[0], 5)
    
    def show_results(self):
        """
        Metodo para exibir os resultados.
        """
        print(f'k: {self.k}')
        print(f'Matriz de confusão:\n{self.confusion_matrix}')
        print(f'Taxa de reconhecimento: {self.accurancy * 100}%')
        print()

for i in [1, 3, 5, 7]:
    Algorithm(i)

k: 1
Matriz de confusão:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa               50                0               0
Iris-versicolor            0               47               3
Iris-virginica             0                3              47
Taxa de reconhecimento: 96.0%

k: 3
Matriz de confusão:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa               50                0               0
Iris-versicolor            0               47               3
Iris-virginica             0                3              47
Taxa de reconhecimento: 96.0%

k: 5
Matriz de confusão:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa               50                0               0
Iris-versicolor            0               47               2
Iris-virginica             0                3              48
Taxa de reconhecimento: 96.667%

k: 7
Matriz de confusão:
                 Iris-setosa  Iris-versicolor  Iris-virginica